# Getting data

The data used in this article is the open sourced dataset, which contains 100K data points, by movielens.
https://grouplens.org/datasets/movielens/

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import sys
sys.path.append('/content/gdrive/My\Drive/Colab\ Notebooks/ml-100k')

In [ ]:
%cd gdrive/My\ Drive/Colab\ Notebooks/ml-100k/

/content/gdrive/My Drive/Colab Notebooks/ml-100k


In [ ]:
import pandas as pd
data = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/ml-100k/u.data",sep='\t',
                      names=['userId', 'movieId', 'rating', 'timestamp'],engine='python')

In [ ]:
data

,userId,movieId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [ ]:
data = data.drop('timestamp',axis=1)

In [ ]:
data

,userId,movieId,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


# Splitting the data

In [ ]:
data.shape

(100000, 3)

In [ ]:
#train_data.shape

In [ ]:
train_data=data.iloc[:int(data.shape[0]*0.80)]
test_data=data.iloc[int(data.shape[0]*0.80):]

In [ ]:
train_data.shape

(80000, 3)

In [ ]:
test_data.shape

(20000, 3)

# SVD MF

In [ ]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 7.3 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1630194 sha256=13f1614fca8c6efd5642b36f9fff480f578c73ecce5d0a1c5d7485b2aaa7ea82
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


http://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD

__ Predicted Rating : __

𝑟̂ 𝑢𝑖=𝜇+𝑏𝑢+𝑏𝑖+𝑞𝑇𝑖𝑝𝑢
𝑞𝑞𝑖 - Representation of item(movie) in latent factor space

𝑝𝑝𝑢 - Representation of user in new latent factor space

A BASIC MATRIX FACTORIZATION MODEL in https://datajobs.com/data-science-repo/Recommender-Systems-[Netflix].pdf
Optimization problem with user item interactions and regularization (to avoid overfitting)
∑𝑟𝑢𝑖∈𝑅𝑡𝑟𝑎𝑖𝑛(𝑟𝑢𝑖−𝑟̂ 𝑢𝑖)2+𝜆(𝑏2𝑖+𝑏2𝑢+||𝑞𝑖||2+||𝑝𝑢||2)

In [ ]:
from surprise import SVD
import numpy as np
import surprise
from surprise import Reader, Dataset

In [ ]:
# train set from train data

In [ ]:
# It is to specify how to read the dataframe.
# for our dataframe, we don't have to specify anything extra..
reader = Reader(rating_scale=(1,5))

# create the traindata from the dataframe...
train_data_mf = Dataset.load_from_df(train_data[['userId', 'movieId', 'rating']], reader)

# build the trainset from traindata.., It is of dataset format from surprise library..
trainset = train_data_mf.build_full_trainset()

In [ ]:
#test set from test data

In [ ]:
# It is to specify how to read the dataframe.
# for our dataframe, we don't have to specify anything extra..
reader = Reader(rating_scale=(1,5))

# create the traindata from the dataframe...
test_data_mf = Dataset.load_from_df(test_data[['userId', 'movieId', 'rating']], reader)

# build the trainset from traindata.., It is of dataset format from surprise library..
testset = test_data_mf.build_full_trainset()

In [ ]:
svd = SVD(n_factors=100, biased=True, random_state=15, verbose=True)
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [ ]:
# storing train predictions

In [ ]:
#getting predictions of trainset
train_preds = svd.test(trainset.build_testset())

train_pred_mf = np.array([pred.est for pred in train_preds])


In [ ]:
# storing test predictions

In [ ]:
#getting predictions of trainset
test_preds = svd.test(testset.build_testset())

test_pred_mf = np.array([pred.est for pred in test_preds])


In [ ]:
train_rmse

NameError: ignored

In [ ]:
train_mape

21.5085126245398

In [ ]:
#pred.shape

In [ ]:
data.shape

(100000, 3)

In [ ]:
test_pred_mf

array([3.18408322, 4.49775307, 3.6064782 , ..., 3.7019292 , 3.94540146,
       3.9708972 ])

In [ ]:
test_pred_mf.shape

(20000,)

In [ ]:
train_pred_mf.shape

(80000,)

In [ ]:
pred=np.concatenate((train_data, test_data))
pred.shape

(100000, 3)

# XGBoost

# Preparing train data frame

In [ ]:
import numpy as np
from scipy import sparse
from scipy.sparse import csr_matrix

In [ ]:
# Creating a sparse matrix
train_sparse_matrix = sparse.csr_matrix((train_data.rating.values, (train_data.userId.values,
                                               train_data.movieId.values)))

In [ ]:
# Global avg of all movies by all users

In [ ]:
train_averages = dict()
# get the global average of ratings in our train set.
train_global_average = train_sparse_matrix.sum()/train_sparse_matrix.count_nonzero()
train_averages['global'] = train_global_average
train_averages

{'global': 3.5315375}

In [ ]:
# get the user averages in dictionary (key: user_id/movie_id, value: avg rating)

def get_average_ratings(sparse_matrix, of_users):

    # average ratings of user/axes
    ax = 1 if of_users else 0 # 1 - User axes,0 - Movie axes

    # ".A1" is for converting Column_Matrix to 1-D numpy array
    sum_of_ratings = sparse_matrix.sum(axis=ax).A1
    # Boolean matrix of ratings ( whether a user rated that movie or not)
    is_rated = sparse_matrix!=0
    # no of ratings that each user OR movie..
    no_of_ratings = is_rated.sum(axis=ax).A1

    # max_user  and max_movie ids in sparse matrix
    u,m = sparse_matrix.shape
    print(sparse_matrix.shape)
    # creae a dictonary of users and their average ratigns..
    average_ratings = { i : sum_of_ratings[i]/no_of_ratings[i]
                                 for i in range(u if of_users else m)
                                    if no_of_ratings[i] !=0}

    # return that dictionary of average ratings
    return average_ratings

In [ ]:
# Average ratings given by a user

In [ ]:
train_averages['user'] = get_average_ratings(train_sparse_matrix, of_users=True)
print('\nAverage rating of user 10 :',train_averages['user'][10])

(944, 1680)

Average rating of user 10 : 4.192771084337349


In [ ]:
# Average ratings given for a movie

In [ ]:
train_averages['movie'] =  get_average_ratings(train_sparse_matrix, of_users=False)
print('\n AVerage rating of movie 15 :',train_averages['movie'][15])

(944, 1680)

 AVerage rating of movie 15 : 3.75


In [ ]:
# get users, movies and ratings from our samples train sparse matrix
train_users, train_movies, train_ratings = sparse.find(train_sparse_matrix)

In [ ]:
#print(train_sparse_matrix)

In [ ]:
print(train_users,train_movies,train_ratings)

[   1    6    8 ... 4682 4751 4790] [   1    1    1 ... 3952 3952 3952] [5 4 4 ... 5 4 3]


In [ ]:
train_users.shape

(800167,)

In [ ]:
train_movies.shape

(800167,)

In [ ]:
train_ratings.shape

(800167,)

In [ ]:
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
final_data = pd.DataFrame()
count = 0
for (user, movie, rating)  in zip(train_users, train_movies, train_ratings):
            st = datetime.now()
            print(user, movie)
            #--------------------- Ratings of "movie" by similar users of "user" ---------------------
            # compute the similar Users of the "user"
            user_sim = cosine_similarity(train_sparse_matrix[user], train_sparse_matrix).ravel()
            top_sim_users = user_sim.argsort()[::-1][1:] # we are ignoring 'The User' from its similar users.
            # get the ratings of most similar users for this movie
            top_ratings = train_sparse_matrix[top_sim_users, movie].toarray().ravel()
            # we will make it's length "5" by adding movie averages to .
            top_sim_users_ratings = list(top_ratings[top_ratings != 0][:5])
            top_sim_users_ratings.extend([train_averages['movie'][movie]]*(5 - len(top_sim_users_ratings)))
        #     print(top_sim_users_ratings, end=" ")


            #--------------------- Ratings by "user"  to similar movies of "movie" ---------------------
            # compute the similar movies of the "movie"
            movie_sim = cosine_similarity(train_sparse_matrix[:,movie].T, train_sparse_matrix.T).ravel()
            top_sim_movies = movie_sim.argsort()[::-1][1:] # we are ignoring 'The User' from its similar users.
            # get the ratings of most similar movie rated by this user..
            top_ratings = train_sparse_matrix[user, top_sim_movies].toarray().ravel()
            # we will make it's length "5" by adding user averages to.
            top_sim_movies_ratings = list(top_ratings[top_ratings != 0][:5])
            top_sim_movies_ratings.extend([train_averages['user'][user]]*(5-len(top_sim_movies_ratings)))
        #     print(top_sim_movies_ratings, end=" : -- ")

            #-----------------prepare the row to be stores in a file-----------------#
            row = list()
            row.append(user)
            row.append(movie)
            # Now add the other features to this data...
            row.append(train_averages['global']) # first feature
            # next 5 features are similar_users "movie" ratings
            row.extend(top_sim_users_ratings)
            # next 5 features are "user" ratings for similar_movies
            row.extend(top_sim_movies_ratings)
            # Avg_user rating
            row.append(train_averages['user'][user])
            # Avg_movie rating
            row.append(train_averages['movie'][movie])

            # finalley, The actual Rating of this user-movie pair...
            row.append(rating)
            count = count + 1
            final_data = final_data.append([row])
            print(count)



            if (count)%10000 == 0:
              #print(','.join(map(str, row)))
              print("Done for {} rows----- {}".format(count, datetime.now()))


Streaming output truncated to the last 5000 lines.
77501
713 1176
77502
720 1176
77503
724 1176
77504
87 1177
77505
405 1177
77506
497 1177
77507
533 1177
77508
222 1178
77509
244 1178
77510
268 1178
77511
279 1178
77512
291 1178
77513
303 1178
77514
314 1178
77515
393 1178
77516
399 1178
77517
405 1178
77518
436 1178
77519
642 1178
77520
660 1178
77521
682 1178
77522
712 1178
77523
804 1178
77524
87 1179
77525
110 1179
77526
222 1179
77527
393 1179
77528
399 1179
77529
405 1179
77530
642 1179
77531
846 1179
77532
125 1180
77533
276 1180
77534
279 1180
77535
378 1180
77536
393 1180
77537
405 1180
77538
476 1180
77539
588 1180
77540
87 1181
77541
279 1181
77542
280 1181
77543
378 1181
77544
393 1181
77545
622 1181
77546
642 1181
77547
660 1181
77548
87 1182
77549
110 1182
77550
279 1182
77551
280 1182
77552
286 1182
77553
303 1182
77554
393 1182
77555
405 1182
77556
417 1182
77557
495 1182
77558
774 1182
77559
846 1182
77560
87 1183
77561
125 1183
77562
194 1183
77563
254 1183
77564
336

In [ ]:
final_data.columns=['user', 'movie', 'GAvg', 'sur1', 'sur2', 'sur3', 'sur4', 'sur5',
            'smr1', 'smr2', 'smr3', 'smr4', 'smr5', 'UAvg', 'MAvg', 'rating']

In [ ]:
final_data.shape

(80000, 16)

In [ ]:
final_data.head()

,user,movie,GAvg,sur1,sur2,sur3,sur4,sur5,smr1,smr2,smr3,smr4,smr5,UAvg,MAvg,rating
0,1,1,3.531538,3.0,4.0,5.0,4.0,5.0,5,5,4,3,4.0,3.597610,3.870423,5
0,2,1,3.531538,1.0,4.0,3.0,5.0,2.0,5,5,4,4,4.0,3.672414,3.870423,4
0,5,1,3.531538,5.0,4.0,3.0,5.0,3.0,4,5,4,3,3.0,2.871166,3.870423,4
0,6,1,3.531538,5.0,4.0,3.0,4.0,4.0,4,2,3,1,5.0,3.641414,3.870423,4
0,10,1,3.531538,4.0,4.0,3.0,3.0,4.0,5,5,4,4,4.0,4.192771,3.870423,4


In [ ]:
final_data['mf_svd']=train_pred_mf

In [ ]:
final_data.head()

,user,movie,GAvg,sur1,sur2,sur3,sur4,sur5,smr1,smr2,smr3,smr4,smr5,UAvg,MAvg,rating,mf_svd
0,1,1,3.531538,3.0,4.0,5.0,4.0,5.0,5,5,4,3,4.0,3.597610,3.870423,5,3.840841
0,2,1,3.531538,1.0,4.0,3.0,5.0,2.0,5,5,4,4,4.0,3.672414,3.870423,4,3.526333
0,5,1,3.531538,5.0,4.0,3.0,5.0,3.0,4,5,4,3,3.0,2.871166,3.870423,4,3.532837
0,6,1,3.531538,5.0,4.0,3.0,4.0,4.0,4,2,3,1,5.0,3.641414,3.870423,4,3.972718
0,10,1,3.531538,4.0,4.0,3.0,3.0,4.0,5,5,4,4,4.0,4.192771,3.870423,4,4.184632


# Preparing test data

In [ ]:
# Creating a sparse matrix
test_sparse_matrix = sparse.csr_matrix((test_data.rating.values, (test_data.userId.values,
                                               test_data.movieId.values)))

In [ ]:
# Global avg of all movies by all users

test_averages = dict()
# get the global average of ratings in our train set.
test_global_average = test_sparse_matrix.sum()/test_sparse_matrix.count_nonzero()
test_averages['global'] = test_global_average
test_averages

{'global': 3.52315}

In [ ]:

# get the user averages in dictionary (key: user_id/movie_id, value: avg rating)

def get_average_ratings(sparse_matrix, of_users):

    # average ratings of user/axes
    ax = 1 if of_users else 0 # 1 - User axes,0 - Movie axes

    # ".A1" is for converting Column_Matrix to 1-D numpy array
    sum_of_ratings = sparse_matrix.sum(axis=ax).A1
    # Boolean matrix of ratings ( whether a user rated that movie or not)
    is_rated = sparse_matrix!=0
    # no of ratings that each user OR movie..
    no_of_ratings = is_rated.sum(axis=ax).A1

    # max_user  and max_movie ids in sparse matrix
    u,m = sparse_matrix.shape
    # creae a dictonary of users and their average ratigns..
    average_ratings = { i : sum_of_ratings[i]/no_of_ratings[i]
                                 for i in range(u if of_users else m)
                                    if no_of_ratings[i] !=0}

    # return that dictionary of average ratings
    return average_ratings




In [ ]:
# Average ratings given by a user

test_averages['user'] = get_average_ratings(test_sparse_matrix, of_users=True)
#print('\nAverage rating of user 10 :',test_averages['user'][10])

# Average ratings given for a movie

test_averages['movie'] =  get_average_ratings(test_sparse_matrix, of_users=False)
print('\n AVerage rating of movie 15 :',test_averages['movie'][15])


 AVerage rating of movie 15 : 3.9183673469387754


In [ ]:
# get users, movies and ratings from our samples train sparse matrix
test_users, test_movies, test_ratings = sparse.find(test_sparse_matrix)

In [ ]:
final_test_data = pd.DataFrame()
count = 0
for (user, movie, rating)  in zip(test_users, test_movies, test_ratings):
            st = datetime.now()
        #     print(user, movie)
            #--------------------- Ratings of "movie" by similar users of "user" ---------------------
            # compute the similar Users of the "user"
            user_sim = cosine_similarity(test_sparse_matrix[user], test_sparse_matrix).ravel()
            top_sim_users = user_sim.argsort()[::-1][1:] # we are ignoring 'The User' from its similar users.
            # get the ratings of most similar users for this movie
            top_ratings = test_sparse_matrix[top_sim_users, movie].toarray().ravel()
            # we will make it's length "5" by adding movie averages to .
            top_sim_users_ratings = list(top_ratings[top_ratings != 0][:5])
            top_sim_users_ratings.extend([test_averages['movie'][movie]]*(5 - len(top_sim_users_ratings)))
        #     print(top_sim_users_ratings, end=" ")


            #--------------------- Ratings by "user"  to similar movies of "movie" ---------------------
            # compute the similar movies of the "movie"
            movie_sim = cosine_similarity(test_sparse_matrix[:,movie].T, test_sparse_matrix.T).ravel()
            top_sim_movies = movie_sim.argsort()[::-1][1:] # we are ignoring 'The User' from its similar users.
            # get the ratings of most similar movie rated by this user..
            top_ratings = test_sparse_matrix[user, top_sim_movies].toarray().ravel()
            # we will make it's length "5" by adding user averages to.
            top_sim_movies_ratings = list(top_ratings[top_ratings != 0][:5])
            top_sim_movies_ratings.extend([test_averages['user'][user]]*(5-len(top_sim_movies_ratings)))
        #     print(top_sim_movies_ratings, end=" : -- ")

            #-----------------prepare the row to be stores in a file-----------------#
            row = list()
            row.append(user)
            row.append(movie)
            # Now add the other features to this data...
            row.append(test_averages['global']) # first feature
            # next 5 features are similar_users "movie" ratings
            row.extend(top_sim_users_ratings)
            # next 5 features are "user" ratings for similar_movies
            row.extend(top_sim_movies_ratings)
            # Avg_user rating
            row.append(test_averages['user'][user])
            # Avg_movie rating
            row.append(test_averages['movie'][movie])

            # finalley, The actual Rating of this user-movie pair...
            row.append(rating)
            count = count + 1
            final_test_data = final_test_data.append([row])
            print(count)



            if (count)%10000 == 0:
                # print(','.join(map(str, row)))
                print("Done for {} rows----- {}".format(count, datetime.now()))


Streaming output truncated to the last 5000 lines.
15002
15003
15004
15005
15006
15007
15008
15009
15010
15011
15012
15013
15014
15015
15016
15017
15018
15019
15020
15021
15022
15023
15024
15025
15026
15027
15028
15029
15030
15031
15032
15033
15034
15035
15036
15037
15038
15039
15040
15041
15042
15043
15044
15045
15046
15047
15048
15049
15050
15051
15052
15053
15054
15055
15056
15057
15058
15059
15060
15061
15062
15063
15064
15065
15066
15067
15068
15069
15070
15071
15072
15073
15074
15075
15076
15077
15078
15079
15080
15081
15082
15083
15084
15085
15086
15087
15088
15089
15090
15091
15092
15093
15094
15095
15096
15097
15098
15099
15100
15101
15102
15103
15104
15105
15106
15107
15108
15109
15110
15111
15112
15113
15114
15115
15116
15117
15118
15119
15120
15121
15122
15123
15124
15125
15126
15127
15128
15129
15130
15131
15132
15133
15134
15135
15136
15137
15138
15139
15140
15141
15142
15143
15144
15145
15146
15147
15148
15149
15150
15151
15152
15153
15154
15155
15156
15157
15158
15159
1

In [ ]:
final_test_data.columns=['user', 'movie', 'GAvg', 'sur1', 'sur2', 'sur3', 'sur4', 'sur5',
            'smr1', 'smr2', 'smr3', 'smr4', 'smr5', 'UAvg', 'MAvg', 'rating']

In [ ]:
final_test_data.shape

(20000, 16)

In [ ]:
test_pred_mf.shape

(20000,)

In [ ]:
final_test_data['mf_svd']=test_pred_mf

# creating XGBoost

In [ ]:
def get_error_metrics(y_true, y_pred):
    rmse = np.sqrt(np.mean([ (y_true[i] - y_pred[i])**2 for i in range(len(y_pred)) ]))
    mae = np.mean(np.abs([(y_true[i] - y_pred[i]) for i in range(len(y_pred)) ]))
    mape = np.mean(np.abs( (y_true - y_pred)/y_true )) * 100
    return rmse, mae, mape

In [ ]:
# prepare train data
x_train = final_data.drop(['user', 'movie','rating'], axis=1)
y_train = final_data['rating']

In [ ]:
# Prepare Test data
x_test = final_test_data.drop(['user','movie','rating'], axis=1)
y_test = final_test_data['rating']

In [ ]:
import xgboost as xgb

In [ ]:

# initialize XGBoost model...
xgb_model = xgb.XGBRegressor(silent=False, n_jobs=13, random_state=15, n_estimators=100)
# dictionaries for storing train and test results
train_results = dict()
test_results = dict()


# fit the model
print('Training the model..')
start =datetime.now()
xgb_model.fit(x_train, y_train, eval_metric = 'rmse')
print('Done. Time taken : {}\n'.format(datetime.now()-start))
print('Done \n')

Training the model..
[07:30:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Done. Time taken : 0:00:04.427985

Done 



In [ ]:


# from the trained model, get the predictions....
print('Evaluating the model with TRAIN data...')
start =datetime.now()
y_train_pred = xgb_model.predict(x_train)
# get the rmse and mape of train data...
rmse_train,mae_train, mape_train = get_error_metrics(y_train.values, y_train_pred)

# store the results in train_results dictionary..
train_results = {'rmse': rmse_train,
                  'mae':mae_train,
                    'mape' : mape_train,
                    'predictions' : y_train_pred}

Evaluating the model with TRAIN data...


In [ ]:
train_results

{'mae': 0.6975433564387262,
 'mape': 27.486615365746115,
 'predictions': array([4.12253  , 3.9976506, 3.6626303, ..., 3.1412084, 1.5323329,
        2.5568175], dtype=float32),
 'rmse': 0.8846175539473663}

In [ ]:

#######################################
# get the test data predictions and compute rmse and mape
print('Evaluating Test data')
y_test_pred = xgb_model.predict(x_test)
rmse_test,mae_test, mape_test = get_error_metrics(y_true=y_test.values, y_pred=y_test_pred)
# store them in our test results dictionary.
test_results = {'rmse': rmse_test,
                  'mae_test' :mae_test,
                    'mape' : mape_test,
                    'predictions':y_test_pred}

Evaluating Test data


In [ ]:
test_results

{'mae_test': 0.7126821204483509,
 'mape': 27.68918161252637,
 'predictions': array([3.8551188, 4.0215044, 3.0502827, ..., 2.426497 , 2.3559992,
        2.9410863], dtype=float32),
 'rmse': 0.8974031626824315}